# Step28, 함수 최적화 

이제 미분을 자동으로 계산 가능  
미분은 다양한 분야에서 다양한 용도로 활용  
그중 가장 중요한 용도는 함수 최적화 

**NOTE_**   
최적화 : 어떤 함수가 주어졌을 때 그 최소값(또는 최댓값)을 반환하는 '입력(함수의 인수)'을 찾는 일  
신경망 학습의 목표 : 손실함수의 출력을 최소화하는 매개변수를 찾는 것이니 최적화 문제에 속한다.

## 28.1 로젠브록 함수 

로젠브록 함수(Rosenbrock function)

$$f(x_0,x_1) = b(x_1 - x_0^2)^2 + (a - x_0)^2$$

a = 1, b = 100 일때 로젠브록 함수가 최솟값이 되는 지점은 (1,1)이다.

## 28.2 미분 계산하기 

로젝브록 함수의 $(x_0,x_1) = (0.0,2.0)$ 에서의 미분 ($\frac{\partial y}{\partial x_0}와 \frac{\partial y}{\partial x_1}$) 계산 

In [1]:
import numpy as np 
from dezero import Variable 

def rosenbrock(x0, x1):
    y = 100 * (x1 - x0**2)**2 + (1 - x0)**2 
    return y

In [3]:
x0 = Variable(np.array(0.0))
x1 = Variable(np.array(2.0))

y = rosenbrock(x0,x1)
y.backward()
print(x0.grad, x1.grad)

-2.0 400.0


이 코드를 실행하면 x0와 x1의 미분은 각각 -2.0과 400.0이라고 나온다.  
이때 두 미분값을 모은 값 (-2.0, 400.0) 벡터 = **기울기(gradient)** or **기울기 벡터**

기울기는 각 지점에서 함수의 출력을 가장 크게 하는 방향을 가리킨다.

## 28.3 경사하강법 구현

복잡한 형상의 함수라면 기울기가 가리키는 방향에 반드시 최대값이 존재한다고 볼수 없다.(마찬가지로 반대 방향에 최솟값이 존재한다고 볼수 없다.)  
그러나 국소적으로 보면 기울기는 함수의 출력을 가장 크게 하는 방향을 나타낸다.

그래서 기울기 방향으로 일정 거리만큼 이동하여 다시 기울기를 구하는 작업을 반복하면 점차 원하는 지점(최대값 혹은 최솟값)에 접근하리라 기대할 수 있다.

이것이 **경사하강법**<sup>gradient descent

In [6]:
x0 = Variable(np.array(0.0))
x1 = Variable(np.array(2.0))
lr = 0.001
iters = 50000

for i in range(iters):
    print(x0,x1)

    y = rosenbrock(x0,x1)

    x0.cleargrad()
    x1.cleargrad()
    y.backward() 

    x0.data -= lr * x0.grad 
    x1.data -= lr * x1.grad 

# Variable 인스턴스를 반복 사용하여 미분값을 구한다.
# 이떄 x0.grad와 x1.grad에는 미분값이 계속 누적되기 때문에 새롭게 미분할 때는 지금까지 누적된 값을 초기화해야한다. 
# 그래서 역전파하기 전에 각 변수의 cleargrad 메서드를 호출하여 미분값을 초기화한것이다.

le(0.9999999992793843) variable(0.9999999985558852)
variable(0.9999999992796722) variable(0.9999999985564618)
variable(0.9999999992799599) variable(0.9999999985570384)
variable(0.9999999992802474) variable(0.9999999985576147)
variable(0.9999999992805348) variable(0.9999999985581907)
variable(0.9999999992808222) variable(0.9999999985587664)
variable(0.9999999992811094) variable(0.999999998559342)
variable(0.9999999992813965) variable(0.9999999985599173)
variable(0.9999999992816834) variable(0.9999999985604924)
variable(0.9999999992819703) variable(0.9999999985610672)
variable(0.999999999282257) variable(0.9999999985616419)
variable(0.9999999992825437) variable(0.9999999985622163)
variable(0.9999999992828302) variable(0.9999999985627905)
variable(0.9999999992831166) variable(0.9999999985633645)
variable(0.9999999992834029) variable(0.9999999985639383)
variable(0.9999999992836891) variable(0.9999999985645118)
variable(0.9999999992839752) variable(0.9999999985650851)
variable(0.99999999928

# Step29, 뉴턴 방법으로 푸는 최적화(수동 계산)

경사하강법은 일반적으로 수렴이 느리다는 단점이 있다.

더 빠른 방법 중 유명한 것 : **뉴런 방법**  
뉴런 방법으로 최적화하면 더 적은 단계로 최적의 결과를 얻을 가능성이 높아진다.

## 29.1 뉴런 방법을 활용한 최적화 이론 

경사하강법 대신 뉴턴 방법을 사용하여 실제로 더 빨리 수렴하는지 확인해 본다.

뉴턴 방법으로 최적화하기  
2차 함수 이용  
y = f(x)라는 함수의 최솟값을 구하는 문제를 생각해보자.  

뉴턴 방법으로 최적화하려면  
테일러 급수에 따라 y = f(x)를 $f(x) = f(a) + f'(a)(x-a) + \frac{1}{2!}f''(a)(x-a)^2 + ...$로 변환한다.  
테일러 급수에 따라 어떤 점 a기점으로 f를 x의 다항식으로 나타낼 수 있다.  
f(x)를 2차 미분항까지만 사용하여 나타내본다. $f(x) \approx f(a) + f'(a)(x-a) + \frac{1}{2}f''(a)(x-a)^2$

즉 y = f(x)는 '어떤 함수'를 x의 2차 함수로 근사한 것이다.

'근사한 2차 함수'는 a에서 'y = f(x)'에 접하는 곡선  

'근사한 2차 함수'를 미분해서 최솟값을 구해본다.
$$\frac{d}{dx}(f(a) + f'(a)(x-a) + \frac{1}{2}f''(a)(x-a)^2 = 0$$
$$f'(a) + f''(a)(x-a) = 0$$
$$x = a - \frac{f'(a)}{f''(a)}$$

'근가한 2차 함수'의 최솟값은 $x = a - \frac{f'(a)}{f''(a)}$ 위치에 있음을 알수 있다.  
즉 a의 위치를 $-\frac{f'(a)}{f''(a)}$ 만큼 갱신하면 된다.

그리고 생신된 a의 위치에서 같은 작업을 반복한다.

경사 하강법 : $x \leftarrow x - \alpha f'(x) $  
뉴턴 방법 : $x \leftarrow x - \frac{f'(x)}{f''(x)}$

경사하강법은 단순히 learning rate(\alpha 라는 계수)를 사람이 수동으로 설정하고 미분값에 곱해주었다. (기울기 방향(1차미분)으로 진행하여 x의 값을 갱신)
  
뉴턴 방법은 2차 미분을 이용하여 경사하강법에서 말하는 \alpha 를 자동으로 조정한다. 즉 $\alpha = \frac{1}{f''(x)}$ 라고 할수 있다.

**CAUTION_** 지금까지 함수의 입력이 '스칼라'일 때의 뉴턴 방법을 설명하였다.  
입력이 '벡터'인 경우로도 자연스럽게 확장할 수 있다.  
벡터인 경우 1차 미분으로 기울기를 사용, 2차 미분으로 허세 행렬(Hessian metrix)를 사용 

## 29.2 뉴턴 방법을 활용한 최적화 구현 

$y = x^4 - 2x^2$는 '오목'한 부분이 2곳이다. 최솟값은 x가 -1과 1인 곳이다. 초기값을 x=2로 설정한 후 최솟값 중 하나인 x=1에 도달할 수 있는지 검증해본다.


DeZero는 2차 미분은 자동으로 구하지 못하므로 다음과 같이 수동으로 2차 미분을 구한다.

$$y=x^4-2x^2$$
$$\frac{\partial y}{\partial x}= 4x^3-4x$$
$$\frac{\partial^2 y}{\partial x^2}=12x^2-4$$

In [8]:
import numpy as np 
from dezero import Variable

def f(x):   # 1차 미분
    y = x ** 4 - 2 * x ** 2 
    return y 

def gx2(x): # 2차 미분 
    return 12 * x ** 2 - 4 

x = Variable(np.array(2.0))
iters = 10 

for i in range(iters):
    print(i,x)

    y = f(x)
    x.cleargrad()
    y.backward() 

    x.data -= x.grad / gx2(x.data)      # 뉴턴 방법 x = x - f'(x)/f''(x)

0 variable(2.0)
1 variable(1.4545454545454546)
2 variable(1.1510467893775467)
3 variable(1.0253259289766978)
4 variable(1.0009084519430513)
5 variable(1.0000012353089454)
6 variable(1.000000000002289)
7 variable(1.0)
8 variable(1.0)
9 variable(1.0)


단 7회 만에 최솟값에 도달하였다.  만약 경사하강법으로 학습률을 0.01로 하고 x=1인 정답과 절대오차가 0.001 이하로 좁혀지기까지 124번 갱신